In [5]:
import util

import jax
import jax.numpy as np

import pandas as pd

import matplotlib.pyplot as plt

In [7]:
import numpy as base_np
from epiweeks import Week, Year
start = '2020-03-15'
forecast_start = '2020-04-26'

num_weeks = 8
data = util.load_state_data()
places = sorted(list(data.keys()))
#places = ['AK', 'AL']

allQuantiles = [0.01,0.025]+list(np.arange(0.05,0.95+0.05,0.05)) + [0.975,0.99]

forecast_date = pd.to_datetime(forecast_start)
currentEpiWeek = Week.fromdate(forecast_date) - 1


forecast = {'quantile':[], 'value':[], 'type':[], 'location':[], 'target':[]}
    
print (currentEpiWeek)
for place in places:
    prior_samples, mcmc_samples, post_pred_samples = util.load_samples(place, path='out')
    forecast_samples = post_pred_samples['z_future']
    t = pd.date_range(start=forecast_start, periods=forecast_samples.shape[1], freq='D')
    
    weekly_df = pd.DataFrame(index=t, data=np.transpose(forecast_samples)).resample("1w",label='left').last()
    weekly_df[weekly_df<0.] = 0.
    for time, samples in weekly_df.iterrows():
        for q in allQuantiles:
                 deathPrediction = base_np.percentile(samples,q*100)
                 forecast["quantile"].append("{:.3f}".format(q))
                 forecast["value"].append(deathPrediction)
                 forecast["type"].append("quantile")
                 forecast["location"].append(place)
                 horizon_date = Week.fromdate(time)
                 week_ahead = horizon_date.week - currentEpiWeek.week
                 forecast["target"].append("{:d} wk ahead cum death".format(week_ahead))
                 currentEpiWeek_datetime = currentEpiWeek.startdate()
                 forecast["forecast_date"] = "{:4d}-{:02d}-{:02d}".format(currentEpiWeek_datetime.year,currentEpiWeek_datetime.month,currentEpiWeek_datetime.day)
                 if q==0.50:
                     forecast["quantile"].append("NA")
                     forecast["value"].append(deathPrediction)
                     forecast["type"].append("point")
                     forecast["location"].append(place)
                     forecast["target"].append("{:d} wk ahead cum death".format(week_ahead))
                     forecast["forecast_date"] = "{:4d}-{:02d}-{:02d}".format(currentEpiWeek_datetime.year,currentEpiWeek_datetime.month,currentEpiWeek_datetime.day)
    #base_np.quantile(hosp,axis=1,q=allQuantiles)

202017


In [8]:

forecast = pd.DataFrame(forecast)


In [9]:
forecast.loc[forecast.type=="point"]


,quantile,value,type,location,target,forecast_date
12,NA,11.549634,point,AK,0 wk ahead cum death,2020-04-19
36,NA,12.489046,point,AK,1 wk ahead cum death,2020-04-19
60,NA,14.279363,point,AK,2 wk ahead cum death,2020-04-19
84,NA,14.814931,point,AK,3 wk ahead cum death,2020-04-19
108,NA,16.278072,point,AK,4 wk ahead cum death,2020-04-19
...,...,...,...,...,...,...
32940,NA,1803.663940,point,WY,22 wk ahead cum death,2020-04-19
32964,NA,2282.117432,point,WY,23 wk ahead cum death,2020-04-19
32988,NA,2593.661133,point,WY,24 wk ahead cum death,2020-04-19
33012,NA,2630.067261,point,WY,25 wk ahead cum death,2020-04-19


In [10]:

fips_codes = pd.read_csv('/Users/gcgibson/covid19-forecast-hub/template/state_fips_codes.csv')


In [11]:
df_truth = forecast.merge(fips_codes, left_on='location', right_on='state', how='left')
df_truth["state_code"] = df_truth["state_code"].astype(int)
df_truth = df_truth[["quantile", "value", "type", "state_code","target","forecast_date"]]

df_truth = df_truth.rename(columns={"state_code": "location"})



In [13]:
import datetime

df_truth['location'] = df_truth['location'].apply(lambda x: '{0:0>2}'.format(x))
df_truth['forecast_date'] =forecast_start
df_truth.to_csv(f'out/sub.csv', float_format="%.0f")
df_truth

,quantile,value,type,location,target,forecast_date
0,0.010,6.549336,quantile,02,0 wk ahead cum death,2020-04-26
1,0.025,6.672437,quantile,02,0 wk ahead cum death,2020-04-26
2,0.050,7.026710,quantile,02,0 wk ahead cum death,2020-04-26
3,0.100,7.963519,quantile,02,0 wk ahead cum death,2020-04-26
4,0.150,8.229877,quantile,02,0 wk ahead cum death,2020-04-26
...,...,...,...,...,...,...
33043,0.850,11587.172786,quantile,56,26 wk ahead cum death,2020-04-26
33044,0.900,13519.323311,quantile,56,26 wk ahead cum death,2020-04-26
33045,0.950,20146.789463,quantile,56,26 wk ahead cum death,2020-04-26
33046,0.975,22507.825098,quantile,56,26 wk ahead cum death,2020-04-26
